## Programa para adquisición en los experimentos de filtros

En todos los experimentos donde estudiamos la respuesta de los filtros, tenemos que hacer un barrido en frecuencias con el generador de funciones y medir sobre la señal de salida con el osciloscopio. En este tuto muestro una guía para hacer la adquisición.

Arrancamos importando las librerías necesarias,

In [1]:
import pyvisa as visa
import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

Defino el objeto resourse manager y listo los instrumentos conectados:

In [ ]:
rm = visa.ResourceManager()

lista_instrumentos = rm.list_resources()  

print(lista_instrumentos)

Busco en la lista al **generador**, copio el nombre en la variable *instrumento_gen* y me conecto:

In [ ]:
instrumento_gen = ''

gen=rm.open_resource()

print(gen.query('*IDN?'))

Idem pero con el **osciloscopio**:

In [ ]:
instrumento_osc = ''

osci=rm.open_resource()

print(osci.query('*IDN?'))

**Importante!** setear la marca correcta del equipo que tienen en la mesa:

In [ ]:
genMarca = '' # poner en esta variable: "Tektronix o "Singlent"

oscMarca = '' # poner en esta variable: "1052B" o  "1000C"

Configuro una onda de salida sinusoidal en el **generador** de amplitud *amp_in* y fase nula:

In [ ]:
amp_in = 5         # amplitud de entrada, permanece cte todo el exp


if genMarca=='Tektronix':

    gen.write("OUTPUT1 OFF")  
    gen.write("SOURCE1:FUNC SIN")
    gen.write("SOURCE1:VOLT %f Vpp" %amp_in) 
    gen.write("OUTPUT1 ON")


if genMarca=='Singlent':

    gen.write('C1:OUTP OFF')  
    gen.write('C1:BSWV WVTP,SINE' ) 
    gen.write('C1:BSWV AMP,%f' % amp_in) 
    gen.write('C1:OUTP ON')



Configuro el **osciloscopio**, formateo de datos y mediciones:

In [ ]:
# Formateo datos de salida
osci.write('DAT:ENC RPB')
osci.write('DAT:WID 1') 
osci.write('HEADER OFF')
osci.write('SEL:CH1 ON')        


if oscMarca == "1052B":
    # Defino medicion 1 de AMPLITUD ch1
    osci.write('MEASU:MEAS1:SOURCE CH1')
    osci.write('MEASU:MEAS1:TYPE CRMS') 

    # Defino medicion 2 de FASE ch1
    osci.write('MEASU:MEAS2:SOURCE CH1')
    osci.write('MEASU:MEAS2:TYPE PHAse')


if oscMarca == "1000C":
    # Defino una UNICA medición inmediata en ch1
    osci.write('MEASUrement:IMMed:SOURce CH1')


Empiezo el **barrido en frecuencias**. Para cada frecuencia voy a medir la amplitud y fase en el salida del filtro, utilizando el CH1:

In [ ]:
frec1 = 500.               # freq mínima
frec2 = 1500.              # freq máxima
N=100                      # can datos

# conjunto de freq del barrido
frecuencias=np.geomspace( frec1, frec2, N ) 


# Defino contenedores para guardar los datos
data={}
data["Freq"]=[]
data["amp_out"]=[]
data["fase_out"]=[]
data["scale_out"]=[]


for freq in frecuencias:

    # SETEO DE FRECUENCIA -----------------
    print (freq)

    if genMarca =='Tektronix':   
        gen.write('SOURCE1:FREQ %f' % freq) 

    if genMarca =='Siglent':
        gen.write('C1:BSWV FRQ,%f' % freq)    


    # SETEO ESTALA OPTIMA EN OSCI ---------
    osci.write('AUTOS EXEC')
    time.sleep(5)


    # CONGELAMOS PANTALLA -----------------
    osci.write('ACQ:STATE OFF')  
    

    # MIDO AMPLITUD Y FASE A LA SALIDA -----

    # tomo la escala para el error
    escalav1 = float(osci.query('CH1:SCAle?'))
    time.sleep(2)

    if oscMarca == '1052B':

        # mido amplitud
        ACH1= float(osci.query('MEASU:MEAS1:VALue?')) 
        time.sleep(2)

        # mido fase
        fase = osci.query_ascii_values('MEASU:MEAS2:VAL?')[0]
        time.sleep(2)


    if oscMarca == '1000C':
        
        # Indico que quiero medir AMPLITUD y mido
        osci.write('MEASUrement:IMMed:TYPe CRMS')
        ACH1= float( osci.query('MEASUrement:IMMed:VALue?') )
        time.sleep(2)

        # Indico que quiero medir FASE y mido
        osci.write('MEASU:IMM:TYP PHAse')
        fase = osci.query_ascii_values('MEASUrement:IMMed:VALue?')[0] 
        time.sleep(2)



    # DES - CONGELAMOS PANTALLA ------------
    osci.write('ACQ:STATE ON')


    # APILO DATOS  -------------------------
    data["Freq"].append( freq )
    data["amp_out"].append( ACH1 )
    data["fase_out"].append( fase )
    data["scale_out"].append( escalav1 )




# GUARDO DATOS 
df = pd.DataFrame( data=data )

df.to_csv( "path/to/you/folder/name.csv", index=False )